In [1]:
# import modules
import pandas as pd
import numpy as np
import torch
import os
import warnings
import pickle
import json
from tqdm import tqdm
from torch.utils.data import Dataset


# HIPS 2021

In [2]:
test_plot_ID=[4351,4370,4371,4390,4391,4410,4411,4430,4431]
def train_test_split(test_plot_ID,output_name): # HIPS_2021
    # test_plot_ID=[4360,4361,4380,4381,4400,4401,4420,4421,4440] # this one is the west of HIPS
     # this is the east of HIPS
    input_folder='/home/tirgan/a/huan1577/Pointnet_Pointnet2_pytorch/data/UAV/HIPS_2021/trim_normalized_fps'
    LAI=pd.read_csv('/home/tirgan/a/huan1577/Pointnet_Pointnet2_pytorch/data/UAV/HIPS_2021/LAI_LiDAR_date.csv',index_col='plot')
    dataset={} 
    dataset['train']=dict()
    dataset['test']={}
    dataset['train']['path']=[]
    dataset['train']['LAI']=[]
    dataset['test']['path']=[]
    dataset['test']['LAI']=[]
    dates=os.listdir(input_folder)
    for date in dates:
        sub_dir=input_folder+'/'+date
        if os.path.isdir(sub_dir)==False: continue
        flist=os.listdir(sub_dir)
        for f in flist:
            fpath=sub_dir+'/'+f
            plot_ID=int(f[9:13]) # for UAV
            # plot_ID=int(f[17:21]) # for bp
            if plot_ID in test_plot_ID:
                dataset['test']['path'].append(fpath)
                dataset['test']['LAI'].append(LAI.loc[plot_ID,str(date)])
            else:
                dataset['train']['path'].append(fpath)
                dataset['train']['LAI'].append(LAI.loc[plot_ID,str(date)])
    # val_plot_ID=[4359,4362,4379,4382,4399,4402,4419,4422,4439]
    # train_plot_ID=[i for i in range(4351,4441)]
    # train_plot_ID=list(set(train_plot_ID)-set(val_plot_ID)-set(test_plot_ID))
    print(dataset['test']['path'][0:2])
    print(dataset['test']['LAI'][0:2])
    print(len(dataset['test']['path']))

    with open(input_folder+'/'+output_name,'w') as output:
        json.dump(dataset,output)
train_test_split(test_plot_ID,'train_test_split.json')

['/home/tirgan/a/huan1577/Pointnet_Pointnet2_pytorch/data/UAV/HIPS_2021/trim_normalized_fps/20210730/20210730_4351_trimmed_normalized_fps.xyz', '/home/tirgan/a/huan1577/Pointnet_Pointnet2_pytorch/data/UAV/HIPS_2021/trim_normalized_fps/20210730/20210730_4370_trimmed_normalized_fps.xyz']
[4.551, 4.489]
45


## 10 folds-straitified by inbred/hybrid

In [3]:
fold0=[4415, 4437, 4413, 4432, 4355, 4375, 4406, 4428, 4360, 4381]
fold1=[4396, 4418, 4370, 4379, 4364, 4392, 4407, 4430, 4411, 4422]
fold2=[4363, 4393, 4369, 4391, 4368, 4377, 4409, 4419, 4403, 4429]
fold3=[4352, 4386, 4397, 4438, 4367, 4376, 4356, 4387, 4412, 4435]
fold4=[4400, 4417, 4366, 4374, 4361, 4389, 4410, 4425]
fold5=[4402, 4431, 4362, 4380, 4359, 4394, 4404, 4423]
fold6=[4408, 4433, 4357, 4383, 4371, 4388, 4405, 4434]
fold7=[4353, 4378, 4395, 4426, 4358, 4373, 4401, 4421]
fold8=[4365, 4382, 4351, 4390, 4399, 4424, 4414, 4420]
fold9=[4372, 4385, 4416, 4427, 4398, 4436, 4354, 4384]
folds=[fold0,fold1,fold2,fold3,fold4,fold5,fold6,fold7,fold8,fold9]
for i in range(10):
    test_plot_ID=folds[i]
    print(test_plot_ID)
    output_name=f'train_test_split_fold{i}.json'
    train_test_split(test_plot_ID,output_name)
    print('-------------------')
    

[4415, 4437, 4413, 4432, 4355, 4375, 4406, 4428, 4360, 4381]
['/home/tirgan/a/huan1577/Pointnet_Pointnet2_pytorch/data/UAV/HIPS_2021/trim_normalized_fps/20210730/20210730_4355_trimmed_normalized_fps.xyz', '/home/tirgan/a/huan1577/Pointnet_Pointnet2_pytorch/data/UAV/HIPS_2021/trim_normalized_fps/20210730/20210730_4360_trimmed_normalized_fps.xyz']
[4.238, 4.012]
50
-------------------
[4396, 4418, 4370, 4379, 4364, 4392, 4407, 4430, 4411, 4422]
['/home/tirgan/a/huan1577/Pointnet_Pointnet2_pytorch/data/UAV/HIPS_2021/trim_normalized_fps/20210730/20210730_4364_trimmed_normalized_fps.xyz', '/home/tirgan/a/huan1577/Pointnet_Pointnet2_pytorch/data/UAV/HIPS_2021/trim_normalized_fps/20210730/20210730_4370_trimmed_normalized_fps.xyz']
[4.792, 4.489]
50
-------------------
[4363, 4393, 4369, 4391, 4368, 4377, 4409, 4419, 4403, 4429]
['/home/tirgan/a/huan1577/Pointnet_Pointnet2_pytorch/data/UAV/HIPS_2021/trim_normalized_fps/20210730/20210730_4363_trimmed_normalized_fps.xyz', '/home/tirgan/a/huan157

## Validatioon set selection (just for testing, you don't need to run this)

In [4]:
from sklearn.model_selection import StratifiedKFold
import re
def validation_set_selector(root,k):
    with open(root+'/'+f'train_test_split_fold{k}.json','r') as json_file:
        data_dict=json.load(json_file)
    date_list=[]
    for i in range(len(data_dict['train']['path'])):
        rlist=re.split('/',data_dict['train']['path'][i])
        date_list.append(rlist[-2])
        
    skf=StratifiedKFold(n_splits=10,random_state=42,shuffle=True)
    skf.get_n_splits(data_dict['train']['path'],date_list)
    for train_index, valid_index in skf.split(data_dict['train']['path'],date_list):
        return train_index,valid_index
        

In [17]:
def validation_set_selector(root,k):
    with open(root+'/'+f'train_test_split_fold{k}.json','r') as json_file:
        data_dict=json.load(json_file)
    stratified_feature=[]
    for i in range(len(data_dict['train']['path'])):
        rlist=re.split('/',data_dict['train']['path'][i])
        plot_ID=int(re.split('_',data_dict['train']['path'][i])[6])
        if plot_ID<4395:
            hi_temp=1
        else:
            hi_temp=0
        stratified_feature.append(rlist[-2]+str(hi_temp))

    skf=StratifiedKFold(n_splits=10,random_state=42,shuffle=True)
    skf.get_n_splits(data_dict['train']['path'],stratified_feature)
    for train_index, valid_index in skf.split(data_dict['train']['path'],stratified_feature):
        return train_index,valid_index

root='/home/tirgan/a/huan1577/Pointnet_Pointnet2_pytorch/data/UAV/HIPS_2021/trim_normalized_fps'
k=0
i,ii=validation_set_selector(root,k)
with open(root+'/'+f'train_test_split_fold{k}.json','r') as json_file:
    data_dict=json.load(json_file)
for j in ii:
    d=re.split('/',data_dict['train']['path'][j])[-2]
    h=int(re.split('_',data_dict['train']['path'][j])[6])
    if h<4395:
        hh='hybrid'
    else:
        hh='inbred'
    print(f'date:{d},hi:{hh}')

date:20210730,hi:hybrid
date:20210730,hi:hybrid
date:20210730,hi:hybrid
date:20210730,hi:hybrid
date:20210730,hi:inbred
date:20210730,hi:inbred
date:20210730,hi:inbred
date:20210730,hi:inbred
date:20210804,hi:hybrid
date:20210804,hi:hybrid
date:20210804,hi:hybrid
date:20210804,hi:hybrid
date:20210804,hi:inbred
date:20210804,hi:inbred
date:20210804,hi:inbred
date:20210804,hi:inbred
date:20210808,hi:hybrid
date:20210808,hi:hybrid
date:20210808,hi:hybrid
date:20210808,hi:hybrid
date:20210808,hi:inbred
date:20210808,hi:inbred
date:20210808,hi:inbred
date:20210808,hi:inbred
date:20210816,hi:hybrid
date:20210816,hi:hybrid
date:20210816,hi:hybrid
date:20210816,hi:hybrid
date:20210816,hi:inbred
date:20210816,hi:inbred
date:20210816,hi:inbred
date:20210816,hi:inbred
date:20210822,hi:hybrid
date:20210822,hi:hybrid
date:20210822,hi:hybrid
date:20210822,hi:hybrid
date:20210822,hi:inbred
date:20210822,hi:inbred
date:20210822,hi:inbred
date:20210822,hi:inbred
